## Taller 2 - Juego del gato con redes neuronales - Parte 2

Ejercicio de construcción de un sistema inteligente con Scikit Learn <br>
Prof. Claudio Morales D. <br>
2019


In [ ]:
#Alumno: 

#### Primera parte: recopilación de datos y entrenamiento de la red neuronal.
Este código no requiere ser modificado.<br>
Basta ejecutarlo sólo una vez para entrenar la red. Luego la red entrenada podrá ser utilizada para hacer predicciones.

In [1]:
import numpy

In [4]:
# gato automatico (autogato)
def display(T):
    print(' '+T[0]+' '+'|'+' '+T[1]+' '+'|'+' '+T[2]+' ')
    print('-----------')
    print(' '+T[3]+' '+'|'+' '+T[4]+' '+'|'+' '+T[5]+' ')
    print('-----------')
    print(' '+T[6]+' '+'|'+' '+T[7]+' '+'|'+' '+T[8]+' ')

def tablero_esta_lleno(T):
    if T.count(' ') > 0:
        return False
    else:
        return True

def espacio_esta_libre(T, pos):
    if T[pos] == ' ':
        return True
    else:
        return False

def es_ganador(T, s):
    return ((T[0]==s and T[1]==s and T[2]==s) or
            (T[3]==s and T[4]==s and T[5]==s) or
            (T[6]==s and T[7]==s and T[8]==s) or
            (T[0]==s and T[3]==s and T[6]==s) or
            (T[1]==s and T[4]==s and T[7]==s) or
            (T[2]==s and T[5]==s and T[8]==s) or
            (T[0]==s and T[4]==s and T[8]==s) or
            (T[2]==s and T[4]==s and T[6]==s))

def juega_x(T):
    buscar = True
    while buscar and not tablero_esta_lleno(T):
        pos = numpy.random.randint(9)
        if espacio_esta_libre(T,pos):
            buscar = False
            T[pos] = 'x'
    return T

def juega_o(T):
    buscar = True
    while buscar and not tablero_esta_lleno(T):
        pos = numpy.random.randint(9)
        if espacio_esta_libre(T,pos):
            buscar = False
            T[pos] = 'o'
    return T

def jugar(T):
    juego = []
    #print('Comienza un nuevo juego...')
    while not tablero_esta_lleno(T):
        if not es_ganador(T, 'o'):
            T = juega_x(T)
            tablero_actual = T[:]
            juego.append(tablero_actual)
            if es_ganador(T,'x'):
                break
        if not es_ganador(T, 'x'):
            T = juega_o(T)
            tablero_actual = T[:]
            juego.append(tablero_actual)
            if es_ganador(T,'o'):
                break
    
    if es_ganador(T, 'x'):
        juego.append('x')
        return(juego)
    elif es_ganador(T, 'o'):
        juego.append('o')
        return(juego)
    else:
        juego.append('E')
        return(juego)
       
# 1. Se reunen datos jugando en forma aleatoria ###############################
log = []
for i in range(2000):
    T0 = [' ' for pos in range(9)]      #tablero inicial
    log.append(jugar(T0))
    
# contabiliza los resultados
gana_x = len([game for game in log if 'x' in game])
gana_o = len([game for game in log if 'o' in game])
empate = len([game for game in log if 'E' in game])

#Imprime resultados
print ("Se jugaron %s juegos:" % len(log))
print ("Gana x: %s veces (%d %%)" % (gana_x, 100*gana_x/len(log)))
print ("Gana o: %s veces (%d %%)" % (gana_o, 100*gana_o/len(log)))
print ("Empate: %s veces (%d %%)" % (empate, 100*empate/len(log)))
print ("----------------------------")

# 2. Acondiciona los datos ###################################################
movimientos =[]
resultados = []
for n in range(len(log)):
    for m in range(1, len(log[n])-1, 2):
            movimientos.append(log[n][m])
            resultados.append(log[n][len(log[n])-1])

# ' ' = 0  ; 'x' = 1  ; 'o' = -1
for n in range(len(movimientos)):
    for m in range(len(movimientos[n])):
        if movimientos[n][m]=='x': movimientos[n][m]=1
        if movimientos[n][m]=='o': movimientos[n][m]=-1
        if movimientos[n][m]==' ': movimientos[n][m]=0

for n in range(len(resultados)):
    if resultados[n]=='x': resultados[n]=1
    if resultados[n]=='o': resultados[n]=-1
    if resultados[n]=='E': resultados[n]=0

# 3. Se construye y se entrena la red neuronal #################################

from sklearn.neural_network import MLPClassifier
red = MLPClassifier(activation='tanh',
                   hidden_layer_sizes=(64,),
                   max_iter=50,
                   learning_rate_init=.1)

# entrenamos la red con el 60% de los datos y probamos con el 40%
n = int(0.6*len(movimientos))
print("Utilizando %s movimientos para entrenar la red" % n)
print ("----------------------------")
red.fit(movimientos[:n], resultados[:n])

#verificamos las predicciones con los datos restantes
esperados = resultados[n:]
prediccion = red.predict(movimientos[n:])

#mostramos resultados
from sklearn import metrics
print("Reporte de predicciones sobre los %s movimientos restantes:" % (len(movimientos)-n))
print("0 = Empate  ; 1 = gana_x  ; -1 = gana_o")
print(" ")
print(metrics.classification_report(esperados, prediccion))

Se jugaron 2000 juegos:
Gana x: 1175 veces (58 %)
Gana o: 597 veces (29 %)
Empate: 228 veces (11 %)
----------------------------
Utilizando 4264 movimientos para entrenar la red
----------------------------
Reporte de predicciones sobre los 2844 movimientos restantes:
0 = Empate  ; 1 = gana_x  ; -1 = gana_o
 
              precision    recall  f1-score   support

          -1       0.53      0.55      0.54       886
           0       0.50      0.59      0.54       500
           1       0.68      0.62      0.65      1458

   micro avg       0.59      0.59      0.59      2844
   macro avg       0.57      0.59      0.58      2844
weighted avg       0.60      0.59      0.59      2844



#### Segunda parte: el juego del gato con red neuronal
El código que se entrega realiza el juego del gato entre dos adversarios automáticos.
Las 'x' juegan en forma aleatoria. Las 'o' juegan utilizando la red neuronal entrenada previamente.<br>
Realice las modificaciones necesarias para hacer que el juego del gato opere ahora en forma inteligente para los dos jugadores, es decir, haga los cambios necesarios para que las 'x'utilicen también las predicciones de la red neuronal ya entrenada para mejorar su juego.<br>
Para esto modifique únicamente la función 'juega_x()', dejando sin cambios el resto del código.<br>
El resultado esperado para este caso es que la mayoría de los juegos terminen en empate.

In [3]:
# gato con red neuronal
def display(T):
    print(' '+T[0]+' '+'|'+' '+T[1]+' '+'|'+' '+T[2]+' ')
    print('-----------')
    print(' '+T[3]+' '+'|'+' '+T[4]+' '+'|'+' '+T[5]+' ')
    print('-----------')
    print(' '+T[6]+' '+'|'+' '+T[7]+' '+'|'+' '+T[8]+' ')

def tablero_esta_lleno(T):
    if T.count(' ') > 0:
        return False
    else:
        return True

def espacio_esta_libre(T, pos):
    if T[pos] == ' ':
        return True
    else:
        return False

def es_ganador(T, s):
    return ((T[0]==s and T[1]==s and T[2]==s) or
            (T[3]==s and T[4]==s and T[5]==s) or
            (T[6]==s and T[7]==s and T[8]==s) or
            (T[0]==s and T[3]==s and T[6]==s) or
            (T[1]==s and T[4]==s and T[7]==s) or
            (T[2]==s and T[5]==s and T[8]==s) or
            (T[0]==s and T[4]==s and T[8]==s) or
            (T[2]==s and T[4]==s and T[6]==s))

#########################################################################
# Realice los cambios sólo en esta parte del código
def juega_x(T):
    buscar = True
    while buscar and not tablero_esta_lleno(T):
        pos = numpy.random.randint(9)
        if espacio_esta_libre(T,pos):
            buscar = False
            T[pos] = 'x'
    return T
##########################################################################

def juega_o(T):
    buscar = True
    while buscar and not tablero_esta_lleno(T):
        # primero buscamos en forma inteligente
        #print('buscando el mejor movimiento...')
        for pos in range(9):
            if espacio_esta_libre(T,pos) and prediccion(T,pos)=='o':
                buscar = False
                T[pos] = 'o'
                #print('se selecciona la posicion ', pos)
                break
        # si no encuentra una buena opción, elige en forma aleatoria
        pos = numpy.random.randint(9)
        if espacio_esta_libre(T,pos):
            buscar = False
            T[pos] = 'o'
            #print('jugando en forma aleatoria')
    return T

def prediccion(T, pos):
    Tn = T[:]
    Tn[pos] = 'o'
    for i in range(len(Tn)):
        if Tn[i] == 'x': Tn[i] = 1
        if Tn[i] == 'o': Tn[i] = -1
        if Tn[i] == ' ': Tn[i] = 0
    p = red.predict([Tn])
    #print(Tn, p)
    if p == [1]: return 'x'
    if p == [-1]: return 'o'
    if p == [0]: return 'E'

def jugar(T):
    juego = []
    #print('Comienza un nuevo juego...')
    while not tablero_esta_lleno(T):
        if not es_ganador(T, 'o'):
            T = juega_x(T)
            tablero_actual = T[:]
            if es_ganador(T,'x'):
                break
        if not es_ganador(T, 'x'):
            T = juega_o(T)
            tablero_actual = T[:]
            if es_ganador(T,'o'):
                break
    
    if es_ganador(T, 'x'):
        juego.append('x')
        return(juego)
    elif es_ganador(T, 'o'):
        juego.append('o')
        return(juego)
    else:
        juego.append('E')
        return(juego)
       

# inicio del juego autogato
log = []
for i in range(100):
    T0 = [' ' for pos in range(9)]      #tablero inicial
    log.append(jugar(T0))

# Imprime los resultados de los juegos
print("En un total de %s juegos:" % len(log))
print("-------------------------")
print("Gana x: %s veces (%d %%)" % (log.count(['x']), 100*log.count(['x'])/len(log)))
print("Gana o: %s veces (%d %%)" % (log.count(['o']), 100*log.count(['o'])/len(log)))
print("Empate: %s veces (%d %%)" % (log.count(['E']), 100*log.count(['E'])/len(log)))

En un total de 100 juegos:
-------------------------
Gana x: 14 veces (14 %)
Gana o: 80 veces (80 %)
Empate: 6 veces (6 %)
